# XQTL worflow master control panel
This notebook provide a master control on the XQTL workflow so it can works on multiple data collection as proposed.
Input:
    A recipe file,each row is a data collection and with the following column:
    
    Theme
        name of dataset, must be different, each uni_study analysis will be performed in a folder named after each, meta analysis will be performed in a folder named as {study1}_{study2}
            
        The column name must contain the # and be the first column
    
    genotype_list
        {Path to file}
    
    molecular_pheno
        {Path to file}
    region_list (list of regions to be analzed)
        {Path to file}
    
    covariate_file
        {Path to file}
    
    factor_analysis_opt
        "APEX" vs "PEER" for factor analysis
    
    LD options:
        "In-sample" LD vs {path to reference panel}
    
    QTL_tool_option
        "APEX" vs "TensorQTL" for QTL association
    
    QTL_analysis_option
        {Int for cis window} vs "trans"
    
    Populations
        The populations from which of the samples was drawn
    
    Conditions:
        The nature of molecular phenotype
    
    ### note: Only data collection from the same Populations and conditions will me merged to perform Fix effect meta analysis
    
Output:
    ...

In [ ]:
Recipe_temp = pd.DataFrame( {"Theme" : ["AC","DLPFC","PCC"] ,
                "genotype_list" : ["/home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt",
                   "/home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt",
                   "/home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt"],
                "molecular_pheno" : ["/home/hs3163/Project/Rosmap/data/gene_exp/AC/geneTpmResidualsAgeGenderAdj_rename.txt",
                     "/home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt",
                     "/home/hs3163/Project/Rosmap/data/gene_exp/PCC/geneTpmResidualsAgeGenderAdj_rename.txt"],
                "region_list" : ["~/GIT/ADSPFG-xQTL/MWE/mwe_region",
                "~/GIT/ADSPFG-xQTL/MWE/mwe_region",
                "~/GIT/ADSPFG-xQTL/MWE/mwe_region"] ,
                "covariate_file" : [".",".","."],
                "factor_analysis_opt" : ["APEX","APEX","APEX"],
                "LD_options": ["In-sample","In-sample","In-sample"],
                "QTL_tool_option" : ["APEX","APEX","APEX"],
                "QTL_analysis_option" : ["cis","cis","cis"],
                "cis_windows" : [500000,500000,5000000],
                "Metal" : ["T","T","F"]})

        ### note: Only data collection from the same Populations and conditions will me merged to perform Fix effect meta analysis
    

In [ ]:
sos dryrun /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control_testing.ipynb data_proc \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" 
    
sos dryrun /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb QTL \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" 

sos dryrun /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb Meta \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" 

sos dryrun /home/hs3163/GIT/ADSPFG-xQTL/workflow/master_control.ipynb Fine_mapping \
        --recipe /home/hs3163/GIT/ADSPFG-xQTL/MWE/mwe_recipe_example \
        --wd ./ \
        --exe_dir "~/GIT/ADSPFG-xQTL/workflow" 


In [ ]:
[global]
## The aforementioned input recipe
parameter: recipe = path
## Overall wd, the file structure of analysis is wd/[steps]/[sub_dir for each steps]
parameter:  wd = path(".")
## Diretory to the excutable
parameter: exe_dir = path("~/GIT/ADSPFG-xQTL/workflow")

parameter: container = '/mnt/mfs/statgen/containers/twas_latest.sif'
parameter: container_lmm = '/mnt/mfs/statgen/containers/lmm.sif'
parameter: container_apex = '/mnt/mfs/statgen/containers/apex.sif'

import pandas as pd
input_inv = pd.read_csv(recipe).to_dict("record")

Metal_theme = input_inv.query("Metal == 'T'")["Theme"].to_list()



## Data Processing
For each theme a new recipe


In [ ]:
[data_proc]
input: for_each = "input_inv"
output: f'{wd}/Data_Processing/{_input["Theme"]}.data_proc_output_recipe.tsv'
task: task: trunk_workers = 1, trunk_size = 1, walltime = '24h',  mem = '40G', tags = f'{step_name}_{_output[0]:bn}'
bash:  expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', volumes = [f'{wd:ad}:{wd:ad}']
    sos run $[exe_dir]/Data_Processing/Data_Processing.ipynb Recipe \
        --molecular_pheno_whole $[_input["molecular_pheno"]] \
        --region_list $[_input["region_list"] \
        --genotype_list $[_input["genotype_list"] \
        --cov $[_input["covariate_file"] \
        --name $[_input["Theme"]] \
        --cointainer_lmm $[cointainer_lmm] \
        --cointainer $[cointainer] \
        --wd $[wd]/Data_Processing/